In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 16467474
paper_name = 'chasse_dohlman_2006' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [76]:
original_data = pd.read_excel('raw_data/Table_S1_copy.xlsx', sheet_name='Table 1')

In [77]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 46 x 6


In [78]:
original_data.head()

,Unnamed: 0,Gene,Phenotype,Unnamed: 3,Gene.1,Phenotype.1
0,1,YOR035C,Group I,47.0,YMR274C,Group IV
1,2,YLR068W,Group I,48.0,YOR299W,Group IV
2,3,YMR116C,Group I,49.0,YOL004W,Group IV
3,4,YJL164C,Group I,50.0,YPL157W,Group IV
4,5,YIL015W,Group I,51.0,YPL138C,Group IV


In [79]:
original_data1 = original_data.iloc[:,[0,1,2]].copy()
original_data2 = original_data.iloc[:,[3,4,5]].copy()

In [80]:
original_data2.columns = original_data1.columns

In [81]:
original_data = pd.concat([original_data1, original_data2], axis=0)

In [82]:
original_data['orf'] = original_data['Gene'].astype(str)

In [83]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [84]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'].values, to='orf')

In [85]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

    Unnamed: 0 Gene Phenotype  orf
44         NaN  NaN       NaN  NAN
45         NaN  NaN       NaN  NAN


In [86]:
original_data = original_data.loc[t,:]

In [87]:
original_data = original_data.reset_index()

In [88]:
original_data['score'] = 1

In [89]:
original_data2 = pd.pivot_table(original_data, index='orf', columns='Phenotype', values='score')

In [90]:
original_data2[original_data2.isnull()] = 0

In [91]:
original_data2.columns

Index(['Group I', 'Group II', 'Group III', 'Group IV', 'Group V', 'Group VI',
       'Groups  I ,VI', 'Groups I, VI'],
      dtype='object', name='Phenotype')

In [92]:
original_data2['Groups I, VI'] = original_data2[['Groups  I ,VI','Groups I, VI']].sum(axis=1)
original_data2.drop(columns=['Groups  I ,VI'], inplace=True)

In [93]:
original_data2['data1'] = original_data2['Group I']*(-0.5) + original_data2['Group II'] * (0.5) + original_data2['Group III'] * (-1) + original_data2['Groups I, VI'] * (-0.5)
original_data2['data2'] = original_data2['Group III']*(-1) + original_data2['Group IV'] * (-0.5) + original_data2['Group V'] * (0.5)
original_data2['data3'] = original_data2['Group VI'] * 1 + original_data2['Groups I, VI'] * 1

In [94]:
original_data2.head()

Phenotype,Group I,Group II,Group III,Group IV,Group V,Group VI,"Groups I, VI",data1,data2,data3
orf,,,,,,,,,,
YAL047C,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0
YBL006C,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.5,0.0
YBR082C,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.5,0.0,0.0
YBR085W,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0
YBR093C,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.5,0.0


In [95]:
original_data2 = original_data2[['data1','data2','data3']]

In [96]:
original_data2 = original_data2.groupby(original_data2.index).mean()

In [97]:
original_data2.shape

(90, 3)

In [98]:
original_data2[original_data2>1] = 1

# Load & process tested strains

In [99]:
tested = pd.read_excel('raw_data/ Gene List_ResGenLibrary.xlsx', sheet_name='mat_a_101501.txt', skiprows=1)

In [100]:
tested.head()

,Unnamed: 0,record no.,ORF name,strain,batch,plate,row,column,RG notes,QC Notes,Unnamed: 10,Unnamed: 11
0,1.0,338,YAL068C,BY4741,01_1,1.0,A,2.0,NaN,NaN,NaN,NaN
1,2.0,339,YAL067C,BY4741,01_1,1.0,A,3.0,NaN,NaN,NaN,NaN
2,3.0,340,YAL066W,BY4741,01_1,1.0,A,4.0,NaN,NaN,NaN,NaN
3,4.0,341,YAL065C,BY4741,01_1,1.0,A,5.0,NaN,NaN,NaN,NaN
4,5.0,345,YAL062W,BY4741,01_1,1.0,A,6.0,NaN,NaN,NaN,NaN


In [101]:
tested['orf'] = tested['ORF name'].astype(str)

In [102]:
tested['orf'] = clean_orf(tested['orf'])

In [103]:
tested.loc[tested['orf']=='YLR287-A','orf'] = 'YLR287C-A'

In [104]:
tested['orf'] = translate_sc(tested['orf'], to='orf')

In [105]:
# Make sure everything translated ok
t = looks_like_orf(tested['orf'])
print(tested.loc[~t,])

Empty DataFrame
Columns: [Unnamed: 0, record no., ORF name, strain, batch, plate, row, column, RG notes, QC Notes, Unnamed: 10, Unnamed: 11, orf]
Index: []


In [106]:
tested_orfs = tested['orf'].unique()

In [107]:
missing = [orf for orf in original_data2.index.values if orf not in tested_orfs]
missing

[]

In [108]:
original_data2 = original_data2.reindex(index=tested_orfs, fill_value=0)

In [109]:
original_data2.head()

Phenotype,data1,data2,data3
orf,,,
YAL068C,0.0,0.0,0.0
YAL067C,0.0,0.0,0.0
YAL066W,0.0,0.0,0.0
YAL065C,0.0,0.0,0.0
YAL062W,0.0,0.0,0.0


# Prepare the final dataset

In [110]:
data = original_data2.copy()

In [111]:
dataset_ids = [11828, 11829, 5180]
datasets = datasets.reindex(index=dataset_ids)

In [112]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [113]:
data.head()

dataset_id,11828,11829,5180
data_type,value,value,value
orf,,,
YAL068C,0.0,0.0,0.0
YAL067C,0.0,0.0,0.0
YAL066W,0.0,0.0,0.0
YAL065C,0.0,0.0,0.0
YAL062W,0.0,0.0,0.0


## Subset to the genes currently in SGD

In [114]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 23


In [115]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,11828,11829,5180
,data_type,value,value,value
gene_id,orf,,,
1869,YAL068C,0.0,0.0,0.0
61,YAL067C,0.0,0.0,0.0
60,YAL066W,0.0,0.0,0.0
1727,YAL065C,0.0,0.0,0.0
57,YAL062W,0.0,0.0,0.0


# Normalize

In [116]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [117]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [118]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,11828,11829,5180,11828,11829,5180
,data_type,value,value,value,valuez,valuez,valuez
gene_id,orf,,,,,,
1869,YAL068C,0.0,0.0,0.0,0.0,0.0,0.0
61,YAL067C,0.0,0.0,0.0,0.0,0.0,0.0
60,YAL066W,0.0,0.0,0.0,0.0,0.0,0.0
1727,YAL065C,0.0,0.0,0.0,0.0,0.0,0.0
57,YAL062W,0.0,0.0,0.0,0.0,0.0,0.0


# Print out

In [119]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [120]:
from IO.save_data_to_db3 import *

In [121]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/3 [00:00<?, ?it/s]

Deleting all datasets for PMID 16467474...
Inserting the new data...


100%|██████████| 3/3 [00:20<00:00,  6.99s/it]

Updating the data_modified_on field...
